In [3]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import os
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
# Set plot style
custom_style = {'figure.facecolor':'.4',
                "axes.facecolor":".4",
                'axes.edgecolor':'.4',
                "axes.labelcolor":"white",
                "axes.grid":True,
                'grid.color':'white',
                "text.color":"white",
                "patch_edgecolor":'white',
                "xtick.color":"white",
                "ytick.color":"white",
                'axes.edgecolor':'black',
               'legend.frameon':True}
sns.set_style('whitegrid')
sns.set_context('poster')
col_palette = sns.color_palette("husl")
gender_palette = sns.color_palette(['#1258DC','#C21460']) # M F
geno_palette = sns.color_palette(['#305A30','#A70F01']) # WT Df1
gengen_palette = sns.color_palette(['#305A30','#A70F01','#559E54','#FE2712']) # WT_M Df1_M WT_F Df1_F
anim_palette = sns.color_palette(['#92C591','#559E54','#305A30','#0A3409','#FE8176','#FE2712','#A70F01','#882216']) # RW7 RW8 RW9 RW10 RZ7 RZ8 RZ9 RZ10
specAnim_palette = sns.color_palette(['#305A30','#0A3409','#A70F01','#882216']) # RW9 RW10 RZ9 RZ10
bandAnim_palette = sns.color_palette(['#92C591','#FE8176','#559E54','#FE2712','#305A30','#A70F01','#0A3409','#882216']) # RW7 RZ7 RW8 RZ8 RW9 RZ9 RW10 RZ10
epoch_palette = sns.color_palette(['#2E2FE3','#FB8604','#448D76']) # Baseline Saline Ketamine
state_palette = sns.color_palette(['#448D76','#FB8604','#EA202C'])
palDict = {'genotype':geno_palette,
           'gender': gender_palette,
           'geno_gender':gengen_palette,
           'animal':anim_palette,
           'epoch_name':epoch_palette,
           'spec_animal':specAnim_palette,
           'state_name':state_palette,
           'band_animal':bandAnim_palette}
groupOrders = {'genotype':['WT','Df1'],
               'gender':['M','F'],
               'geno_gender':['WT_M','Df1_M','WT_F','Df1_F'],
               'animal':['RW7','RW8','RW9','RW10','RZ7','RZ8','RZ9','RZ10'],
               'epoch_name':['Baseline','Saline','Ketamine'],
               'spec_animal':['RW9','RW10','RZ9','RZ10'],
               'state_name':['NREM','Rest','Active'],
               'band_animal':['RW7','RZ7','RW8','RZ8','RW9','RZ9','RW10','RZ10']}
def addTitle(fig=None,title='',adj=0.85):
    plt.subplots_adjust(top=adj)
    fig.fig.suptitle(title)

In [5]:
traceList = []
specList = []
bandList = []
project_path = '/data/Projects/'
figure_dir = project_path+'rn_Schizophrenia_Project/SFN2018_Figures/'

for dirpath,dirnames,files in os.walk(project_path):
    for name in files:
        if name.endswith('deltaTraceMatrix.csv'):
            traceList.append(os.path.join(dirpath+'/'+name))
        if name.endswith('sleepSpecMatrix.csv'):
            specList.append(os.path.join(dirpath+'/'+name))
        if name.endswith('sleepBandPowerMatrix.csv'):
            bandList.append(os.path.join(dirpath+'/'+name))

In [6]:
exclusions = ('state_name=="Artifact"',
              'state_name=="Transition"',
              'state_name=="REM"',
              'animal=="RW9" and (state_name=="Active" or tetrode==6)', # allow Active in trace plots
              'animal=="RW7" and day==5',
              'animal=="RZ7" and day==3',
              'animal=="RZ9" and day==5 and tetrode==8',
              'epoch_name=="Home"')
qStr = 'not ('+') and not ('.join(exclusions)+')'
traceExclusions = ('state_name=="Artifact"',
              'animal=="RW9" and tetrode==6', # allow Active, REM and Transition in trace plots
              'animal=="RW7" and day==5',
              'animal=="RZ7" and day==3',
              'animal=="RZ9" and day==5 and tetrode==8',
              'epoch_name=="Home"',
              'gender=="F"')
tqStr = 'not ('+') and not ('.join(traceExclusions)+')'

In [7]:
traceMat = pd.DataFrame()
#specMat = pd.DataFrame()
#bandMat = pd.DataFrame()
for rf in traceList:
    tmp = pd.read_csv(rf)
    traceMat = traceMat.append(tmp)
#for rf in specList:
#    tmp = pd.read_csv(rf)
#    specMat = specMat.append(tmp)
#for rf in bandList:
#    tmp = pd.read_csv(rf)
#    bandMat = bandMat.append(tmp)
traceMat = traceMat.reset_index(drop=True)
#bandMat = bandMat.reset_index(drop=True)
#specMat = specMat.reset_index(drop=True)
print('Data Loaded!')

Data Loaded!


In [8]:
# Change Epoch Names (Baseline, Saline, Ketamine, Home)
# Change Band Names (Delta, Theta)
bandDict = {'delta':'Delta','theta':'Theta'}
#bandMat['power_band']=bandMat['power_band'].map(bandDict)
epochDict = {'Sleep':'Baseline','Saline':'Saline','Ketamine':'Ketamine','Home':'Home'}
traceMat['epoch_name'] = traceMat['epoch_type'].map(epochDict)
#specMat['epoch_name'] = specMat['epoch_type'].map(epochDict)
#bandMat['epoch_name'] = bandMat['epoch_type'].map(epochDict)

# Make epoch timestamps
traceMat['epoch_timestamp'] = pd.to_datetime(traceMat['epoch_time'],unit='s')
traceMat['epoch_minutes'] = traceMat['epoch_time']/60

# Make geno_gender column
#specMat['geno_gender'] = specMat['genotype'].map(str) + '_' + specMat['gender']
#bandMat['geno_gender'] = bandMat['genotype'].map(str) + '_' + bandMat['gender']

In [9]:
# Add state names
stateDict = {1:'REM',2:'NREM',3:'Rest',4:'Active',5:'Transition',6:'Artifact'}
#bandMat['state_name']=bandMat['sleep_state'].map(stateDict)
#specMat['state_name']=specMat['sleep_state'].map(stateDict)
traceMat['state_name']=traceMat['sleep_state'].map(stateDict)

In [11]:
# Exclude
tmpDat = traceMat.query(tqStr)
#bandDat = bandMat.query(qStr)
#specDat = specMat.query(qStr)

# At each timepoint average over tetrodes
avgDat = traceMat.groupby(['animal','genotype','day','epoch_name','epoch_time','epoch_timestamp','epoch_minutes']).mean().reset_index().drop('tetrode',axis=1).sort_values(by=['animal','day','epoch_name','epoch_time'])

# Group and bin and smooth traces
# TODO: Smooth
traceDat = tmpDat.groupby(
    ['animal','genotype','day','epoch_name']
    )['epoch_timestamp','epoch_time','epoch_minutes','age','sleep_state',
      'STFT_delta_power','CWT_delta_power'].resample('1T',on='epoch_timestamp').mean()
traceDat = traceDat.reset_index()

In [12]:
traceDat = traceDat.loc[:,['animal','genotype','day','epoch_name','epoch_time','epoch_minutes','age','CWT_delta_power']]
traceDat.head()

,animal,genotype,day,epoch_name,epoch_time,epoch_minutes,age,CWT_delta_power
0,RW10,WT,1,Baseline,30.0,0.5,143.0,0.789855
1,RW10,WT,1,Baseline,90.0,1.5,143.0,0.694583
2,RW10,WT,1,Baseline,150.0,2.5,143.0,0.637130
3,RW10,WT,1,Baseline,210.0,3.5,143.0,1.038654
4,RW10,WT,1,Baseline,270.0,4.5,143.0,0.822435


In [14]:
def time_bin(x):
    if x<=5:
        return 1
    if x<=10:
        return 2
    if x<=15:
        return 3
    if x<=20:
        return 4
    if x<=25:
        return 5
    if x<=30:
        return 6
    if x>30:
        return 7
traceDat['Time Group'] = traceDat['epoch_minutes'].apply(time_bin)

In [16]:
baseline = traceDat[traceDat.epoch_name=='Baseline']
other = traceDat[traceDat.epoch_name!='Baseline']
for name, data in other.groupby(['Time Group','genotype']):
    tmp1 = baseline[baseline.genotype==name[1]]
    

(1.0, 'Df1')
(1.0, 'WT')
(2.0, 'Df1')
(2.0, 'WT')
(3.0, 'Df1')
(3.0, 'WT')
(4.0, 'Df1')
(4.0, 'WT')
(5.0, 'Df1')
(5.0, 'WT')
(6.0, 'Df1')
(6.0, 'WT')
(7.0, 'Df1')
(7.0, 'WT')
